In [1]:
! pip install transformers[sentencepiece] datasets tokenizers evaluate
! pip install torch
! pip install tensorflow
! pip install spacy
! pip install seqeval
! pip install ipywidgets
! pip install "ray[tune]" scipy sklearn
! pip install pyinterval

  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2342 sha256=388a79d45e31d528cf4d0cf8da143208772ab4025b1b63c7847be4f5056f9773
  Stored in directory: /Users/breaux/Library/Caches/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn
     |████████████████████████████████| 1.6 MB 2.8 MB/s eta 0:00:01
  Created wheel for pyinterval: filename=pyinterval-1.2.0-py3-none-any.whl size=11239 sha256=59b8050719377824378f493f4a2b0c1c42cdaf49b49dbe2e46b1e0d476b64c26
  Stored in directory: /Users/breaux/Library/Caches/pip/wheels/f0/fc/af/8c69a2ed7df2bb1aeaa5fb056b77bc943fb0af605fdfed3937
  Created wheel for crlibm: filename=crlibm-1.0.3-cp38-cp38-macosx_10_9_x86_64.whl size=128925 sha256=ae93ed77359a20ad4a12c39b667f4e91ef6e435e319af6b24ca23d2d36598c68
  Stored in directory: /Users/breaux/Library/Caches/pip/wheels/0c/94/29/3e0e876a557418f2b0dbcfe5b6e65d8922297ad991f314927b
Successfully built pyinterval crlibm


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/mobile_privacy/cleaned/models

Mounted at /content/drive
/content/drive/MyDrive/mobile_privacy/cleaned/models


In [2]:
import json
from interval import interval
from copy import deepcopy
from random import shuffle
from functools import cmp_to_key
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_sm")
from spacy.training import offsets_to_biluo_tags, biluo_to_iob

In [3]:
def replace_linebreak(text):
  return text.replace("\r","")

def load_jsonl_file(filename):
  '''
  load the original jsonl file.
  replace the line break \r\n to \n, in order to get correct label offset.
  return a list of dict{'text':string, 'label':list of [start, end, category]}
  '''
  jsonl_content = open(filename,'r').read()
  result = [json.loads(jline) for jline in jsonl_content.splitlines()]
  for data in result:
    data['text'] = replace_linebreak(data['text'])
  return result

In [4]:
def overlapped(label1, label2):
  ''' 
  Input: two labels with [start, end, category]
  Output: whether they are overlapped
  Notice that the range of token is [start, end), end is exclusive
  '''
  start1, end1 = label1[0], label1[1]-1
  start2, end2 = label2[0], label2[1]-1
  r1 = interval([start1, end1])
  r2 = interval([start2, end2])
  if len((r1 & r2)) == 0:
    return False
  return True

In [5]:
url_df = pd.read_csv('./app_name_category_300.csv', encoding='latin',on_bad_lines = 'warn')
id_app = {}
for index, row in url_df.iterrows():
  id_app[row['scenario_id']] = [row['app_name'], row['app_category'], row['scenario_id']]

TypeError: read_csv() got an unexpected keyword argument 'on_bad_lines'

In [12]:
jsonl_file = load_jsonl_file('./raw_300_scenarios.jsonl')

In [54]:
from re import I

label_match_dict = {
    'O':'O',
    'B-Noun Phrase':'B-SIM',
    'I-Noun Phrase': 'I-SIM',
    'B-Complex Terms':'B-COM',
    'I-Complex Terms':'I-COM',
    'B-Questions':'B-QUE',
    'I-Questions':'I-QUE'
}

def find_overlap(label1, label_list):
  for i in range(len(label_list)):
    if overlapped(label1, label_list[i]):
      return i
  return -1

def dedup_scenario_label(jsonl_list):
  '''
  Deduplication rules:
  If noun phrase conflict with clause, keep clause only.
  If clause conflict with each other, keep the longest one.
  If noun conflict with each other, keep the shortest one.
  '''
  res = []
  for scenario in jsonl_list:
    tmp = {}
    tmp['id'] = scenario['id']
    tmp['text'] = scenario['text']
    tmp['app_url'] = scenario['app_url']
    tmp['scenario_id'] = scenario['scenario_id']
    tmp['raw_label'] = deepcopy(scenario['label'])
    tmp['app_name'] = id_app[scenario['scenario_id']][0]
    tmp['app_category'] = id_app[scenario['scenario_id']][1]
    labels = list()
    noun = list()
    clause = list()
    for label in scenario['label']:
      start, end, category = label
      if category == 'Noun Phrase':
        noun.append(label)
      else:
        clause.append(label)
    # add non-conflict clause first
    for label in clause:
      ind = find_overlap(label, labels)
      if ind == -1:
        labels.append(label)
      else:
        if labels[ind][1] - labels[ind][0] < label[1] - label[0]:
          labels[ind] = label
    # dedup noun
    dedup_noun = list()
    for label in noun:
      ind = find_overlap(label, dedup_noun)
      if ind == -1:
        dedup_noun.append(label)
      else:
        if dedup_noun[ind][1] - dedup_noun[ind][0] > label[1] - label[0]:
          dedup_noun[ind] = label
    # add non-conflict noun
    for label in dedup_noun:
      ind = find_overlap(label, labels)
      if ind == -1:
        labels.append(label)
    tmp['label'] = labels
    res.append(tmp)
  return res


def convert_jsonl_to_bio(jsonl_list):
  res = {}
  for data in jsonl_list:
    id = data['scenario_id']
    clean_text = data['text']
    raw_labels = deepcopy(data['label'])
    labels = data['label']
    doc = nlp(clean_text)
    biluo_tags = offsets_to_biluo_tags(doc, labels)
    iob_tags = biluo_to_iob(biluo_tags)
    codes = list()
    for c in iob_tags:
        codes.append(label_match_dict.get(c,'O'))
    res[id] = {'id':id,
               'text':clean_text,
               'words': [str(token) for token in doc],
               'codes':codes,
               'raw_labels': raw_labels,
               'app_name':data['app_name'],
               'app_category':data['app_category']}
  return res

dedup_jsonl_file = dedup_scenario_label(jsonl_file)
bio_jsonl = convert_jsonl_to_bio(dedup_jsonl_file)

/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "The goals that I want to achieve through this scre..." with entities "[[94, 123, 'Questions'], [805, 846, 'Questions'], ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "When I open up the Twitch app and navigate to the ..." with entities "[[590, 630, 'Complex Terms'], [898, 964, 'Complex ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be ali

In [55]:
with open('./scenarios-labeled.json', 'w') as f:
  f.write(json.dumps(bio_jsonl))

In [13]:
label_map = {}
replace = {'Noun Phrase': 'SIM', 'Questions': 'QUE', 'Complex Terms': 'COM'}
for d in jsonl_file:
    for label in d['label']:
        label[2] = replace[label[2]]
    label_map[d['scenario_id']] = d['label']
json.dump(label_map, open('../datasets/labels.json', 'w'))